<a href="https://colab.research.google.com/github/ruiborreicho/RedeNeural/blob/main/redeNeuralv1ButBetter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Rede Neural**

## **Colheita de um dataset de imagens a ser utilizado na rede neural que pretendemos desenvolver**

Criação da diretoria em que serão armazenados os dados do dataset de nuImages, seguido da transferência do mesmo e a descompressão dos dados na diretoria anteriormente criada.

In [ ]:
!mkdir -p /nuimages  # Make the directory to store the nuImages dataset in.

!wget https://www.nuscenes.org/data/nuimages-v1.0-mini.tgz  # Download the nuImages mini split.

!tar -xf nuimages-v1.0-mini.tgz -C /nuimages  # Uncompress the nuImages mini split.

#!pip install nuscenes-devkit &> /dev/null

Criação da diretoria onde serão armazenadas todas as imagens.

In [ ]:
!mkdir -p /imagens

Importações das bibliotecas que irão ser utilizadas

In [ ]:
import glob
import shutil
import os

Definição da diretoria de origem dos ficheiros

In [ ]:
src_dir = "/nuimages"

Definição da diretoria de destino dos ficheiros

In [ ]:
dst_dir = "/imagens"

Instruções desenvolvidas que permitem a cópia de todos os ficheiros de formato .JPG da diretoria de origem para a diretoria de destino, que foram declaradas e definidas anteriormente.

In [ ]:
!rm -rf /imagens/* #Delete all files located in imagens directory

In [ ]:
for root, _, files in os.walk(src_dir, topdown=True):
   for name in files:
      if name.endswith('.jpg') :
        file_path = os.path.join(root, name)
        shutil.copy(file_path, dst_dir)

Instalação de dependências

In [ ]:
!pip install --upgrade google-api-python-client
!pip install google-cloud
!pip install google-cloud-vision

Importação da biblioteca do Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Cópia das imagens para um diretório no Google Drive

In [ ]:
!cp -av "/imagens" "/gdrive/MyDrive/IMAGENS"

## **Criação rede neural**

### **Instalação de bibliotecas necessárias**

Instalação do CUDA, CUDNN e Tensorflow

In [ ]:
# Check libcudnn8 version
!apt-cache policy libcudnn8

# Install latest version
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

# Export env variables
!export PATH=/usr/local/cuda-11.4/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-11.4/lib64:$LD_LIBRARY_PATH
!export LD_LIBRARY_PATH=/usr/local/cuda-11.4/include:$LD_LIBRARY_PATH
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI/lib64

# Install tensorflow
!pip install tflite-model-maker==0.4.0
!pip uninstall -y tensorflow && pip install -q tensorflow==2.9.1
!pip install pycocotools==2.0.4
!pip install opencv-python-headless==4.6.0.66

Verificação da instalação da biblioteca tensorflow

In [ ]:
%tensorflow_version 2.x
!pip show tensorflow

Instalação da biblioteca tf_slim

In [ ]:
!pip install tf_slim

Instalação da biblioteca tensorflow_io

In [ ]:
!pip install tensorflow_io

Instalação da biblioteca tf-models-official

In [ ]:
!pip install -U tf-models-official

Instalação da biblioteca tensorflow-io

In [ ]:
!pip install tensorflow-io

Instalação da biblioteca tensorflow-object-detection-api

In [ ]:
!pip install tensorflow-object-detection-api

Instalação da biblioteca LVIS

In [ ]:
!pip install lvis

Clonagem do repositório Tensorflow Models

In [ ]:
%cd /root/
!git clone https://github.com/tensorflow/models.git

Instalação do Tensorboard

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

Configuração do ambiente

In [ ]:
%cd /root/models/research/
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'
!python object_detection/builders/model_builder_test.py

Conclusão da instalação do Tensorboard

In [ ]:
#the logs that are created while training 
LOG_DIR = "/root/models/trained"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### **Coleção de imagens, criação do das labels nos dados e criação do mapa das labels**

O primeiro passo efetuado nesta etapa foi a coleção de imagens para a criação de um dataset. Para realizar essa tarefa, utilizamos o código demonstrado na primeira parte deste Colab Notebook, em que fomos transferir as imagens do dataset nuImages para um Google Drive.

De seguida, utilizamos um programa denominado de labelImg, que pode ser instalado numa máquina Windows (que possua uma versão recente de Python e o Pip), com o comando demonstrado abaixo, e utilizamos esse programa para anotar as imagens com quais os objetos que queremos que este referencie, a localização dos mesmos e o seu respetivo nome.

In [ ]:
!pip3 install labelImg #o comando deve ser corrido sem o "!" na máquina local

O dataset que foi transferido na sua totalidade possui 465 imagens, logo, e com a finalidade de testar a rede neural e conseguir que a mesma esteja a funcionar, decidimos começar por selecionar apenas uma parte das imagens colecionadas, neste caso 49 imagens que compõem o dataset "sample" do nuImages.

Depois da anotação de cada imagem, é criado um ficheiro XML que fornece as informações acerca das anotações que foram efetuadas, sendo que iremos fornecer um exemplo de um desses ficheiros de seguida.

In [ ]:
#este é o exemplo de um dos ficheiros XML gerados, que teve de ser apropriamente anotado pois não se trata de uma linguagem permitida pelo Colab
#<annotation>
#	<folder>images</folder>
#	<filename>3.jpg</filename>
#	<path>C:\Users\rborreic\Desktop\Projects\RedeNeural\images\3.jpg</path>
#	<source>
#		<database>Unknown</database>
#	</source>
#	<size>
#		<width>1600</width>
#		<height>900</height>
#		<depth>3</depth>
#	</size>
#	<segmented>0</segmented>
#	<object>
#		<name>car</name>
#		<pose>Unspecified</pose>
#		<truncated>0</truncated>
#		<difficult>0</difficult>
#		<bndbox>
#			<xmin>638</xmin>
#			<ymin>419</ymin>
#			<xmax>706</xmax>
#			<ymax>469</ymax>
#		</bndbox>
#	</object>
#	<object>
#		<name>van</name>
#		<pose>Unspecified</pose>
#		<truncated>0</truncated>
#		<difficult>0</difficult>
#		<bndbox>
#			<xmin>772</xmin>
#			<ymin>409</ymin>
#			<xmax>862</xmax>
#			<ymax>502</ymax>
#		</bndbox>
#	</object>
#</annotation>

Dando por terminadas as anotações no nosso dataset de teste, iremos então proceder ao upload dos ficheiros para o Colab. Para simplificar este processo iremos clonar as imagens, que foram entretanto depositadas no repositório GitHub do projeto para a pasta do tensorflow models que foi clonada anteriormente.

In [ ]:
#!rm -rf /root/models/RedeNeural/ #Delete all files located in RedeNeural directory
#!rm -r /root/models/RedeNeural #Delete RedeNeural folder

In [ ]:
%cd /root/models/
!git clone https://github.com/ruiborreicho/RedeNeural

### **Criação de um label map**

Criação de um label map, uma representação de todos os objetos que expectamos encontrar dentro das anotações criadas (ficheiros XML)

In [ ]:
labels = [{'name':'car', 'id':1}, {'name':'truck', 'id':2}, {'name':'motorbike', 'id':3}, {'name':'bus', 'id':4}, {'name':'van', 'id':5}, {'name':'bike', 'id':6}]

In [ ]:
%cd /root/models/RedeNeural/annotations
with open('labelMap.pbtxt', 'w') as f:
  for label in labels:
    f.write('item{\n')
    f.write('\tname:\'{}\'\n'.format(label['name']))
    f.write('\tid:{}\n'.format(label['id']))
    f.write('}\n')

### **Criação dos TFRecords**

Utilização do ficheiro generate_tfrecord.py, fornecido pelo tutorial oficial do Object Detection API, que permite a conversão dos dados para o formato requirido pelo modelo.

In [ ]:
!python {'/root/models/RedeNeural/tfrecord/generate_tfrecord.py'} -x {'/root/models/RedeNeural/images/train'} -l {'/root/models/RedeNeural/annotations/labelMap.pbtxt'} -o {'/root/models/RedeNeural/annotations/train.record'}
!python {'/root/models/RedeNeural/tfrecord/generate_tfrecord.py'} -x {'/root/models/RedeNeural/images/test'} -l {'/root/models/RedeNeural/annotations/labelMap.pbtxt'} -o {'/root/models/RedeNeural/annotations/test.record'}

### **Transferência e setup do SSD MobileNet v2**

Para realizarmos o treino do nosso modelo iremos utilizar um modelo já existente para ajudar no processo, sendo o escolhido o SSD MobileNet v2. Nos próximos passos iremos fazer a transferência, movimentação do ficheiro transferido e a descompressão do mesmo.

Iremos agora criar uma diretoria com o nome de "my_ssd_mobnet" onde iremos copiar o ficheiro de pipeline.config do modelo já existente.

In [ ]:
!mkdir -p /root/models/my_ssd_mobnet
!cp {'/root/models/RedeNeural/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {'/root/models/my_ssd_mobnet/'}

### **Update no ficheiro de config para a transferência do conhecimento**

Antes de darmos por inicio ao update do ficheiro, serão necessárias importar algumas dependências.

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

Configuração do caminho até ao ficheiro de config

In [ ]:
CONFIG_PATH = '/root/models/my_ssd_mobnet/pipeline.config'

Defenition of the training parameters on the pipeline.config file

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 

In [ ]:
pipeline_config.model.ssd.num_classes = 6
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = '/root/models/RedeNeural/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = '/root/models/RedeNeural/annotations/labelMap.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = ['/root/models/RedeNeural/annotations/train.record']
pipeline_config.eval_input_reader[0].label_map_path = '/root/models/RedeNeural/annotations/labelMap.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = ['/root/models/RedeNeural/annotations/test.record']

Escrita dos parametros no ficheiro pipeline.config

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)  

### **Treino do modelo**

Comando que inicia o treino do modelo, sendo que é possível aumentar o número de train_steps para que o treino seja mais aprofundado.

In [ ]:
!python /root/models/research/object_detection/model_main_tf2.py --model_dir=/root/models/my_ssd_mobnet --pipeline_config_path=/root/models/my_ssd_mobnet/pipeline.config --num_train_steps=5000

Exportação do modelo

In [ ]:
!python /root/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir /root/models/my_ssd_mobnet/ \
    --output_directory /root/models/ \
    --pipeline_config_path /root/models/my_ssd_mobnet/pipeline.config

Carregamento do modelo

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap('/root/models/RedeNeural/annotations/labelMap.pbtxt', use_display_name=True)
model = tf.saved_model.load('/root/models/saved_model')

### **Teste do modelo**

Importação das bibliotecas necessárias

In [ ]:
import cv2 
import numpy as np
from six import BytesIO
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
from object_detection.builders import model_builder
from object_detection.utils import ops as utils_ops

Função para carregar uma imagem para um NumPy Array

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path (this can be local or on colossus)

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

Função para correr a inferência numa imagem singular

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Correr a inferência

In [ ]:
images = ['/root/models/RedeNeural/images/test/17.jpg', '/root/models/RedeNeural/images/test/20.jpg', '/root/models/RedeNeural/images/test/46.jpg', '/root/models/RedeNeural/images/test/5.jpg']

for image_name in images:
  
  image_np = load_image_into_numpy_array(image_name)
  output_dict = run_inference_for_single_image(model, image_np)

  vis_utils.visualize_boxes_and_labels_on_image_array(
                image_np,
                output_dict['detection_boxes'],
                output_dict['detection_classes'],
                output_dict['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)
  display(Image.fromarray(image_np))